In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import cred as c

## Praw is specifically for reddit api
import praw

print(praw.__version__)

7.7.1


In [2]:
reddit = praw.Reddit(
client_id = c.client_id,
client_secret  = c.client_secret,
user_agent = c.agent
)

In [6]:
submission_list = []
author_list = []

subreddit = reddit.subreddit('ukpolitics').new(limit=10)

for submission in subreddit:
    
    # Extracting the submission id for extracting the comments
    submission_data = reddit.submission(id=submission.id)
    submission_data.comments.replace_more(limit=None)
    comments = [(comment.body, comment.score) for comment in submission_data.comments.list()]
    
    ## getting the author data and storing it in different database 
    author = submission_data.author
    
    if author:
        author_data = {
            'name': author.name,
            'id': author.id,
            'is_mod': author.is_mod,
            'is_employee': author.is_employee,
            'is_gold': author.is_gold,
            'link_karma': author.link_karma,
            'comment_karma': author.comment_karma,
            'created_utc': author.created_utc,
            'verified': author.has_verified_email,
            'profile_picture': author.icon_img,
        }
    else:
        author_data = {
            'name': None,
            'id': None,
            'is_mod': None,
            'is_employee': None,
            'is_gold': None,
            'link_karma': None,
            'comment_karma': None,
            'created_utc': None,
            'verified': None,
            'profile_picture': None,
        }
    
    author_list.append(author_data)
        
    submission_info = {
        'title': submission.title,
        'id': submission.id,  # foreign key 
        'author': str(submission.author),  # foreign key 
        'selftext': submission.selftext,
        'score': submission.score,
        'created_utc': submission.created_utc,
        'upvote_ratio': submission.upvote_ratio,
        'num_comments': submission.num_comments,
        'subreddit': str(submission.subreddit),
        'url': submission.url,
        'over_18': submission.over_18,
        'edited': submission.edited,
        'num_crossposts': submission.num_crossposts,
        'permalink': submission.permalink,
        'comments': comments
    }
    submission_list.append(submission_info)

# Convert the list of dictionaries to a DataFrame
df_submission = pd.DataFrame(submission_list)
df_author = pd.DataFrame(author_list)

# Display the DataFrames
print(df_author)
print(df_submission)


                  name         id  is_mod  is_employee  is_gold  link_karma  \
0             laci4225     11llz7   False        False    False         943   
1  Dawnbringer_Fortune   lki3xvyv    True        False    False       31789   
2           wappingite      gveuu    True        False    False       96795   
3               dumael      3d0w8   False        False    False         659   
4             ProxyZee   dipnosj9    True        False    False       33734   
5          MajorHubbub  u0589llb5   False        False    False       29663   
6            bloomberg   1095cgk5    True        False    False      394653   
7      FormerlyPallas_     16kg1n    True        False    False      161372   
8               paolog      3lgfn    True        False    False        7241   
9      FormerlyPallas_     16kg1n    True        False    False      161372   

   comment_karma   created_utc  verified  \
0             64  1.474588e+09      True   
1          44720  1.649146e+09      True  

In [12]:
df_author.head()

,name,id,is_mod,is_employee,is_gold,link_karma,comment_karma,created_utc,verified,profile_picture
0,laci4225,11llz7,False,False,False,943,64,1.474588e+09,True,https://styles.redditmedia.com/t5_cp135/styles...
1,Dawnbringer_Fortune,lki3xvyv,True,False,False,31789,44720,1.649146e+09,True,https://i.redd.it/snoovatar/avatars/nftv2_bmZ0...
2,wappingite,gveuu,True,False,False,96795,192704,1.402060e+09,True,https://www.redditstatic.com/avatars/defaults/...
3,dumael,3d0w8,False,False,False,659,22630,1.233087e+09,True,https://styles.redditmedia.com/t5_b6h0r/styles...
4,ProxyZee,dipnosj9,True,False,False,33734,1870,1.627239e+09,True,https://styles.redditmedia.com/t5_4t3ixw/style...


In [9]:
df_submission.head()

,title,id,author,selftext,score,created_utc,upvote_ratio,num_comments,subreddit,url,over_18,edited,num_crossposts,permalink,comments
0,Sooo; when did this anti-woke/pro-woke debate ...,1e069s6,laci4225,I am just really curious when did the people o...,0,1.720644e+09,0.50,3,ukpolitics,https://www.reddit.com/r/ukpolitics/comments/1...,False,1720644160.0,0,/r/ukpolitics/comments/1e069s6/sooo_when_did_t...,[(When they ran out of any actual policy ideas...
1,NEW: Labour government to put failing water co...,1e03fqs,Dawnbringer_Fortune,-banning bonuses for worst polluters\n- potent...,193,1.720637e+09,0.99,13,ukpolitics,https://x.com/pickardje/status/181110148026044...,False,False,0,/r/ukpolitics/comments/1e03fqs/new_labour_gove...,[(Snapshot of _NEW: Labour government to put f...
2,Dawn Butler apparently reported to police for ...,1e03fcj,wappingite,,8,1.720637e+09,0.67,37,ukpolitics,https://www.theguardian.com/politics/article/2...,False,False,0,/r/ukpolitics/comments/1e03fcj/dawn_butler_app...,[(Snapshot of _Dawn Butler apparently reported...
3,The Unpopular ‘Popular Conservatives’ Blame th...,1e03e02,dumael,,11,1.720637e+09,0.76,7,ukpolitics,https://bylinetimes.com/2024/07/10/popular-con...,False,False,0,/r/ukpolitics/comments/1e03e02/the_unpopular_p...,[(Snapshot of _The Unpopular ‘Popular Conserva...
4,UK politics: Badenoch confirms she did critici...,1e02r29,ProxyZee,,5,1.720635e+09,0.70,1,ukpolitics,https://www.theguardian.com/politics/live/2024...,False,False,0,/r/ukpolitics/comments/1e02r29/uk_politics_bad...,[(Snapshot of _UK politics: Badenoch confirms ...


In [13]:
df_author.columns

Index(['name', 'id', 'is_mod', 'is_employee', 'is_gold', 'link_karma',
       'comment_karma', 'created_utc', 'verified', 'profile_picture'],
      dtype='object')

In [14]:
df_submission.columns

Index(['title', 'id', 'author', 'selftext', 'score', 'created_utc',
       'upvote_ratio', 'num_comments', 'subreddit', 'url', 'over_18', 'edited',
       'num_crossposts', 'permalink', 'comments'],
      dtype='object')

In [18]:
df_author.to_csv('author_data_reddit.csv', encoding='utf-8',index = False)

In [19]:
df_submission.to_csv('submission_data_reddit.csv', encoding='utf-8', index = False)

# Progress :

### The submission data and the corresponding author data has been recorded. 
### The sample datasets have been created. Updated on github